In [5]:
# imports 

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [8]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")



API key found and looks good so far!


In [9]:
openai = OpenAI()

In [10]:
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [45]:
web = Website("https://www.t-mobile.com/cell-phone-plans?INTNAV=tNav%3APlans%3AMagenta")
##print(web.text)

In [46]:
system_prompt = (
    "You are a tool designed to analyze websites and provide detailed information "
    "about the prices and benefits of each plan mentioned on the site."
)


In [47]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt = (
            f"You are looking at a website titled '{website.title}'.\n"
            "The contents of this website are as follows:\n"
            "Please provide a short summary of this website in markdown format.\n"
            "Additionally, if applicable, please include the mobile plans offered, their prices with autopay if applicable, and the number of lines available.\n\n"
            f"{website.text}"
        )
    return user_prompt

In [48]:
# See how this function creates exactly the format above
def messages_for(website):
    return [
        { "role": "system", "content": system_prompt},
        { "role": "user", "content": user_prompt_for(website)}
    ]

In [49]:
# And now: call the OpenAI API. You will get very familiar with this!

def mobilePlans(url):
    website = Website(url)
    response = ollama.chat(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response['message']['content']

In [50]:
# A function to display this nicely in the Jupyter output, using markdown

def displayMobilePlans(url):
    # Generate the combined summary for both websites
    combined_summary = summarize(url)
    
    # Display the combined summary in Markdown format
    display(Markdown(combined_summary))
    

In [54]:
displayMobilePlans("https://www.t-mobile.com/cell-phone-plans?INTNAV=tNav%3APlans%3AMagenta")

# Summary of "Our Best Unlimited Data Cell Phone Plans: Compare Go5G Plan Deals"

This website is dedicated to offering an overview of T-Mobile's unlimited data cell phone plans. It emphasizes helping users compare various plans, highlighting competitive pricing, benefits, and features compared to other major carriers like AT&T and Verizon.

## Mobile Plans Offered

### 1. **Essentials Saver**
- **Price:**
  - 1 line: $55
  - 2 lines: $90
  - 3 lines: $115
- **Features:**
  - Unlimited talk and text
  - 50GB premium data
  
### 2. **Go5G Next**
- **Price:**
  - 1 line: $105
  - 2 lines: $180
  - 3 lines: $230
  - 4 lines: $280
  - 5 lines: $330
- **Features:**
  - Upgrade-ready every year
  - Unlimited premium data
  - Includes streaming services (Hulu, Apple TV+, Netflix) on eligible plans.

### 3. **Go5G Next First Responder / Military / 55+**
- **Price:**
  - 1 line: $90
  - 2 lines: $140
  - 3 lines: $180
  - 4 lines: $220
  - 5 lines: $260
  - 6-12 lines: $50-$55/line
- **Features:**
  - Same as Go5G Next with a discount for first responders, military personnel, and seniors.

### 4. **Magenta MAX**
- **Price:**
  - 1 line: $90
  - 2 lines: $150
  - 3 lines: $185
  - 4 lines: $220
  - 5 lines: $255
- **Features:**
  - Unlimited premium data, Netflix included.

### 5. **Magenta**
- **Price:**
  - 1 line: $75
  - 2 lines: $130
  - 3 lines: $155
  - 4 lines: $180
- **Features:**
  - 100GB premium data, Netflix included.

### 6. **Essentials 55+**
- **Price:**
  - 1 line: $50
  - 2 lines: $70
- **Features:**
  - Unlimited talk and text
  - 50GB premium data

### 7. **Magenta Puerto Rico**
- **Price:**
  - 1 line: $65
  - 2 lines: $110
  - 3 lines: $155
- **Features:**
  - 100GB premium data, Netflix included.

### 8. **Magenta Military/First Responder**
- **Price:**
  - 1 line: $60
  - 2 lines: $90
- **Features:**
  - 100GB premium data, Netflix included.

## Key Benefits of All Plans
- Unlimited 5G & 4G LTE data
- Nationwide 5G coverage
- Advanced scam-blocking protection
- No annual contracts

This collection of plans ensures flexibility for individuals, families, seniors, first responders, and military members, catering to diverse needs while providing competitive pricing in the market.